---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open('university_towns.txt') as f:
        wiki_raw = f.readlines()

    lst = []
    
    for line in wiki_raw:
        if "[edit]" in line:
            state = line[:-7]
        else:
            #city = re.sub(r"\(.*\)", "", line)
            #city = re.sub(r"\(.*\]", "", city)
            city = re.sub(r"\(.*$", "", line)
            #city = re.sub(r"\[.*\]", "", city)
            city = city.strip()
            lst.append((state,city))

    df = pd.DataFrame.from_records(lst, columns=['State','RegionName'])
            
    return(df)
#get_list_of_university_towns()

In [4]:
def get_gdp():
    gdp = pd.read_excel(open('gdplev.xls','rb'), sheetname=0, skiprows=5, skip_footer=0)
    gdp = gdp.drop('Unnamed: 0', 1)
    gdp = gdp.drop('GDP in billions of current dollars', 1)
    gdp = gdp.drop('GDP in billions of chained 2009 dollars', 1)
    gdp = gdp.drop('Unnamed: 3', 1)
    gdp = gdp.drop('GDP in billions of current dollars.1', 1)
    gdp = gdp.drop('Unnamed: 7', 1)
    gdp.columns = ['Quarter','GDP']
    small = gdp[gdp['Quarter'] >= '2000q1']
    small.set_index(['Quarter'], inplace=True)
    # this code courtesy of https://stackoverflow.com/questions/41361537/find-two-consecutive-quarters-of-gdp-decline-and-ending-with-two-consecutive-qu
    # I'm somewhat uncomfortable with my classmates asking exact questions like this on public sites. Bad form.
    small['change'] = small.diff()        # change in GDP from prior quarter

    # if the current change is negative, and the previous change is negative, we are in recession
    start = (small.change<0) & (small.change.shift(-1)<0)  # potential start
    # if the current change is positive, and the previus change is positive, we are out
    end   = (small.change>0) & (small.change.shift(-1)>0)  # potential end
    #print(end)
    # this tracks our current status, -1 means in recession, +1 means growth
    small['recess' ] = np.nan
    small.loc[ start, 'recess' ] = -1
    small.loc[ end,   'recess' ] =  1

    small['recess'] = small.recess.ffill()    # if the current row doesn't fit the 
                                              # definition of a start or end, then  
                                              # fill it with the prior row value                                           
    small['startend'] = np.nan
    small.loc[ (small.recess==-1) & (small.recess.shift() == 1), 'startend'] = -1  # start
    small.loc[ (small.recess== 1) & (small.recess.shift(1)==1) & (small.recess.shift(2)==-1), 'startend'] =  1  # end # problem MUST be here!!

    return(small)
#print(get_gdp())

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    
    recs=gdp[gdp.startend.notnull()]
    
    return recs.iloc[0].name
#get_recession_start()

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    
    recs=gdp[gdp.startend.notnull()]       
    
    return recs.iloc[1].name # why is this off by one?
#get_recession_end()

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    
    recs=gdp[gdp.startend.notnull()]  

    # this feels Pythonic
    recession = gdp.loc[get_recession_start(): get_recession_end()]  
    #gdp.loc[gdp['Quarter'].idxmax()]
    return recession.loc[recession['GDP'].idxmin()].name
#get_recession_bottom()

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
    house['State'] = house['State'].map(lambda x: states[x])
    columns_to_keep = ['RegionName',
                   'State']
    data_columns = [x for x in list(house) if (x >= '2000-01') & (x <= '2016-12')]
    comb = columns_to_keep + data_columns
    house = house[comb]
    house.set_index(['State','RegionName'], inplace=True)
    final = house.groupby(pd.PeriodIndex(house.columns, freq='Q'), axis=1).mean()
    return final
#convert_housing_data_to_quarters()

In [28]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    # I struggle with PeriodIndex, this is a big kludge
    house = convert_housing_data_to_quarters()
    start = get_recession_start()
    start_house = house[start]
    start_house.name = 'start'
    bottom = get_recession_bottom()
    bottom_house = house[bottom]
    bottom_house.name = 'bottom'
    h = pd.concat([start_house, bottom_house],axis=1)
    h.reset_index(inplace=True)
    # now we have just the two quarters we need
    university_towns = get_list_of_university_towns()
    u = university_towns.reset_index()
    hdf = pd.merge(h, u, on=['State','RegionName'], how='left', indicator='Exist')
    hdf = hdf.drop('index',1)
    # then compute decline/growth
    hdf['PriceRatio'] = hdf['start'].div(hdf['bottom'])
    hdf.set_index(['State','RegionName'], inplace=True)
    # if it exists in the university table, separate it
    uni = hdf[(hdf['Exist'] == 'both') & pd.notnull(hdf['PriceRatio'])]
    non = hdf[(hdf['Exist'] == 'left_only') & pd.notnull(hdf['PriceRatio'])]
    # get means
    uni_mean = uni['PriceRatio'].mean()
    non_mean = non['PriceRatio'].mean()

    # run the ttest
    t,p = ttest_ind(uni['PriceRatio'], non['PriceRatio'])
    
    if (p < 0.01):
        different = True
    else:
        different = False
    
    if (uni_mean < non_mean):
        better = 'university town'
    else:
        better = 'non-university town'
        
    return (different,p,better)

#run_ttest()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/

(True, 0.0054964273536938875, 'university town')